In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import concurrent.futures
from csv import reader

In [2]:
kolesa_list = []
urls = []
n_pages = 0
for i in range(1,1001):
    n_pages += 1
    url = 'https://kolesa.kz/cars/?page='
    page = url + str(i)
    urls.append(page)
    
for url in urls:
    print(url)
    break

https://kolesa.kz/cars/?page=1


In [3]:
def get_data(url):
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 OPR/72.0.3815.320 (Edition Campaign 34)'
    }
    r = requests.get(url, headers=headers)

    # Convert to Beautiful Soup Object
    soup = bs(r.content, 'lxml')
    
    # Добавляем отдельно Бренд Модель и Год
    car_name = soup.find(class_='offer__title')
    cars = {}
    if car_name.find('span', itemprop='name'):
        cars['Бренд'] = car_name.find('span', itemprop='brand').get_text()
        cars['Модель'] = car_name.find('span', itemprop='name').get_text()
        cars['Год'] = car_name.find('span', class_='year').get_text()
    else:
        cars['Бренд'] = car_name.find('span', itemprop='brand').get_text()
        cars['Год'] = car_name.find('span', class_='year').get_text()

    # Добавляем колонку отдельно
    info_car = soup.find(class_='offer__sidebar-info')
    info_cars = info_car.find_all('dl')
    for row in info_cars:
        if row.find('span').get_text() not in cars:
            cars[row.find('span').get_text()] = row.find('dd').get_text(' ', strip=True)

    # Добавляем колонку Цены Машины
    price_cars = soup.find_all('div', class_="offer__sidebar-header")
    for row in price_cars:
        s = row.find('div', class_='offer__price')
        cars['Цена'] = s.get_text(" ", strip=True).replace('\xa0', ' ')
    
    # Добавляем колонку Средней цены Данной Машины
    mean_price = soup.find_all('div', class_='kolesa-score-average')
    for row in mean_price:
        s = row.find('div', class_='kolesa-score-price')
        cars['Средння цена'] = s.get_text('', strip=True).replace('\xa0', ' ')

    return cars

In [4]:
def get_all_data(urls):
    for url in urls:
        r = requests.get(url)
        soup = bs(r.content)
        kolesa = soup.select('.a-el-info-title a')
        kolesa_list = []
        basic_path = 'https://kolesa.kz'
        for index, car in enumerate(kolesa):
            try:
                relative_path = car['href']
                full_path = basic_path + relative_path
                kolesa_list.append(get_data(full_path))
            except Exception as e:
                print(car['href'])
                print(e)

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(get_all_data, urls)

In [ ]:
kolesa_list